In [ ]:
%pip install langchain chromadb pypdf
%pip install chromadb
%pip install langchain-text-splitters
%pip install langchain-core
%pip install langchain-community

In [2]:
# from langchain.document_loaders import PyPDFLoader

In [18]:
# local_path = '../docs/240-core-java-interview-questions-and-answers.pdf'

# # Local PDF file uploads
# pdf_loader = PyPDFLoader(local_path)
# documents = pdf_loader.load()
# documents[0]

# Specify the local path to your text file
local_path = '../docs/java.txt'

# Load the text file
with open(local_path, 'r', encoding='utf-8') as file:
    text_content = file.read()

# Display the first few lines or the entire content
print(text_content[:500])

Question                 | Difficulty Level
---------------------------------------------------------------------
What is a class in Java? | 1
How do you define a method in Java? | 1
What is the purpose of the main method in Java? | 1
What is a variable in Java? | 1
How do you declare an array in Java? | 1
What is the difference between int and float in Java? | 1
What is the syntax for a for loop in Java? | 1
How do you create an object in Java? | 1
What is a constructor in Java? | 1
What is the


In [19]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [20]:
!ollama list

NAME                       ID              SIZE      MODIFIED      
nomic-embed-text:latest    0a109f422b47    274 MB    3 seconds ago    


In [21]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [28]:
# Define the Document class
class Document:
    def __init__(self, content, metadata=None):
        self.page_content = content
        self.metadata = metadata if metadata is not None else {}

In [29]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents([Document(text_content)])

In [30]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="microservice-rag"
)

OllamaEmbeddings: 100%|██████████| 13/13 [00:37<00:00,  2.91s/it]

: 

In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [2]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [3]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [4]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

NameError: name 'vector_db' is not defined